<a href="https://colab.research.google.com/github/leoninekev/ml-engine-custom-prediction-routine/blob/master/custom_prediction_model_deployment_Walkthrough.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ## Custom prediction routines 

**Model deployment task in google Ai-latform necessitates compliance, completion of following three sub-rotines:**
* Upload of model artifacts to cloud storage bucket (Artifacts include - saved model/keras Model file, custom prediction, pre/post processing scripts, confi files)
* Creation of ai-platform **Model Resource**.
* Creation of ai-platform **Version Resource** (Specifying path to cloud storage holding Model & artifact package)

###  Custom prediction routines determines, what code runs when an online prediction request to ai-platform is made.

Deploying a custom prediction routine as **Version Resource** serves many utilities as follows:
* It enables AI Platform to run a custom python code in response to each incoming request received for prediction/inferencing.
* It allows preprocessing of input data before it is forwarded to a trained model for prediction.
* It also allows postprocessing model's output prior to actual posting of prediction result, thus modifying output to suit any application's endpoints requirements.

In [38]:
import sys

if 'google.colab' in sys.modules:
  from google.colab import auth as google_auth
  google_auth.authenticate_user()

# If you are running this notebook locally, replace the string below with the
# path to your service account key and run this cell to authenticate your GCP
# account.
else:
  %env GOOGLE_APPLICATION_CREDENTIALS ''


W0730 10:14:37.331681 140175376680832 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [43]:
PROJECT_ID = "nifty-episode-231612" #@param {type:"string"}
! gcloud config set project $PROJECT_ID

!gcloud config list

Updated property [core/project].
[component_manager]
disable_update_check = True
[core]
account = quantumbisht@gmail.com
project = nifty-episode-231612

Your active configuration is: [default]


In [33]:
!git clone https://github.com/leoninekev/ml-engine-custom-prediction-routine.git
  
%cd ml-engine-custom-prediction-routine/

! ls -pR

Cloning into 'ml-engine-custom-prediction-routine'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 49 (delta 20), reused 41 (delta 18), pack-reused 0
Unpacking objects: 100% (49/49), done.
/content/ml-engine-custom-prediction-routine
.:
config.pickle					      resnet.py
custom_prediction_model_deployment_Walkthrough.ipynb  roi_helpers.py
FixedBatchNormalization.py			      RoiPoolingConv.py
LICENSE						      setup.py
predictor.py					      test_img.jpg
README.md					      test_json_1706.json


* View the setup file and edit the package name and version if needed.

In [0]:
%pycat setup.py

* **Packaging the Predictor module and other supporting artifacts/Scripts.**

In [25]:
!python setup.py sdist --formats=gztar

running sdist
running egg_info
creating test_code_new_model.egg-info
writing test_code_new_model.egg-info/PKG-INFO
writing dependency_links to test_code_new_model.egg-info/dependency_links.txt
writing requirements to test_code_new_model.egg-info/requires.txt
writing top-level names to test_code_new_model.egg-info/top_level.txt
writing manifest file 'test_code_new_model.egg-info/SOURCES.txt'
writing manifest file 'test_code_new_model.egg-info/SOURCES.txt'
running check


creating test_code_new_model-0.1
creating test_code_new_model-0.1/test_code_new_model.egg-info
copying files to test_code_new_model-0.1...
copying FixedBatchNormalization.py -> test_code_new_model-0.1
copying LICENSE -> test_code_new_model-0.1
copying README.md -> test_code_new_model-0.1
copying RoiPoolingConv.py -> test_code_new_model-0.1
copying config.pickle -> test_code_new_model-0.1
copying custom_prediction_model_deployment_Walkthrough.ipynb -> test_code_new_model-0.1
copying predictor.py -> test_code_new_model-0.

* **Copy the model file/model weights and packaged python artifacts to model directory in Cloud storage bucket; The path of which will be used to cite trained model during Version resource creation.**

* Since the frcnn model was trained in ml-engine before in a different notebook, and the model weights were hosted on dropbox.
* downloading model_weights file here to later copy custom code package & model weights together to  model directory in cloud storage bucket.

In [12]:
!wget -O model_frcnn.hdf5 https://www.dropbox.com/s/makxzi5aoe1bfij/model_frcnn.hdf5?dl=0

--2019-07-30 09:36:32--  https://www.dropbox.com/s/makxzi5aoe1bfij/model_frcnn.hdf5?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/makxzi5aoe1bfij/model_frcnn.hdf5 [following]
--2019-07-30 09:36:32--  https://www.dropbox.com/s/raw/makxzi5aoe1bfij/model_frcnn.hdf5
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc95ac0a175fb68a88392ed889e8.dl.dropboxusercontent.com/cd/0/inline/AloRlg89m03ZhwnY0aCAONKf1gj3mtDYITywVum4uKfYLVgkECb0uKEhFxbhw-2Ip87yOECYQuBHcO829oIW0rcUuK5v3Dot10C4jw3z5auCcA/file# [following]
--2019-07-30 09:36:32--  https://uc95ac0a175fb68a88392ed889e8.dl.dropboxusercontent.com/cd/0/inline/AloRlg89m03ZhwnY0aCAONKf1gj3mtDYITywVum4uKfYLVgkECb0uKEhFxbhw-2Ip87yOECYQuBHcO829oIW0rcUuK5v3Dot10C4jw3z5auCcA/file
Re

In [26]:
!ls -pR

.:
config.pickle
custom_prediction_model_deployment_Walkthrough.ipynb
dist/
FixedBatchNormalization.py
LICENSE
model_frcnn.hdf5
predictor.py
README.md
resnet.py
roi_helpers.py
RoiPoolingConv.py
setup.py
test_code_new_model.egg-info/

./dist:
test_code_new_model-0.1.tar.gz

./test_code_new_model.egg-info:
dependency_links.txt  PKG-INFO	requires.txt  SOURCES.txt  top_level.txt


*  **Copy model_drcnn.hdf5 and test_code-0.1.tar.gz to model directory(gs://nifty-episode-231612-mlengine/cloud_test_package_repo/)**

In [27]:
!gsutil cp dist/test_code_new_model-0.1.tar.gz gs://nifty-episode-231612-mlengine/cloud_test_package_repo/

!gsutil cp model_frcnn.hdf5 gs://nifty-episode-231612-mlengine/cloud_test_package_repo/

Copying file://dist/test_code_new_model-0.1.tar.gz [Content-Type=application/x-tar]...
-
Operation completed over 1 objects/13.2 KiB.                                     
Copying file://model_frcnn.hdf5 [Content-Type=application/octet-stream]...
\
Operation completed over 1 objects/108.6 MiB.                                    


**Run following to Create a Model resource**
* Define model name and Region.
* Also Enable Online prediction logging, to stream logs that contain the **stderr and stdout streams** from your prediction nodes, It proves useful for debugging during version creation and inferencing.

In [18]:
MODEL_NAME = "FoodPredictor_app"
REGION='asia-northeast1'

! gcloud beta ai-platform models create $MODEL_NAME \
  --regions $REGION --enable-console-logging

Created ml engine model [projects/nifty-episode-231612/models/FoodPredictor_app].


In order to create Version resource to serve predictions, **Ensure the following:**
* The model file or model weights file, model config are stored in a model directory (example here, **gs://nifty-episode-231612-mlengine/cloud_test_package_repo/** in Cloud Storage.
* The implementation of the predictor interface and other dependencies packaged as custom code previously is also staged in the same model directory.

  The structure of your version resource directory in cloud storage bucket therefore should be as:

     ``.your-bucket-name/cloud_test_package_repo/``  
     ``model_frcnn.hdf5``   
     ``model.config(if any)``
     ``test_code-0.1.tar.gz``

The ``test_code-0.1.tar.gz`` holds the custom pre/post processing code and most importantly the``predictor`` module, for AI Platform to access ``predictor.MyPredictor``.

* The **`--trace-log`** flag lets you view Version resource creation logs in the cell below (One can
also view logs and other job details in the GCP Console, if you've enbaled **Stackdriver logging service**.)


Now to further with creating Version Resource; Ensure following crucial parameters are defined:
* **model** = $MODEL_NAME (your model name from Model Resource)
* **VERSION_NAME** = 'v1_a' (your version name)
* **python-version** = 3.5 (Python version for which the custom code is written)
* **runtime-version**= 1.5 (Tensorflow's runtime version)
* **origin** = gs://nifty-episode-231612-mlengine/cloud_test_package_repo/ (model directory holdinig model, model artifacts & packaged code)
* **package-uris**=  gs://nifty-episode-231612-mlengine/cloud_test_package_repo/test_code_new_model-0.1.tar.gz (your packaged custom predictor interface code)
* **prediction-class**=  predictor.MyPredictor (Call to main predictor class)

In [0]:
MODEL_NAME="FoodPredictor_app"
VERSION_NAME='v1_a'

!gcloud beta ai-platform versions create $VERSION_NAME --model $MODEL_NAME --python-version 3.5 --runtime-version 1.5 --machine-type mls1-c4-m2 --origin gs://nifty-episode-231612-mlengine/cloud_test_package_repo/ --package-uris gs://nifty-episode-231612-mlengine/cloud_test_package_repo/test_code_new_model-0.1.tar.gz --prediction-class predictor.MyPredictor --trace-log

In [29]:
!gcloud logging logs list

NAME
projects/nifty-episode-231612/logs/cloudaudit.googleapis.com%2Factivity
projects/nifty-episode-231612/logs/master-replica-0
projects/nifty-episode-231612/logs/ml.googleapis.com%2Fprimary.stderr
projects/nifty-episode-231612/logs/ml.googleapis.com%2Fprimary.stdout
projects/nifty-episode-231612/logs/ml.googleapis.com%2Ftest_job_GcloudColab
projects/nifty-episode-231612/logs/ml.googleapis.com%2Ftest_job_GcloudColab_1
projects/nifty-episode-231612/logs/ml.googleapis.com%2Ftest_job_GcloudColab_2
projects/nifty-episode-231612/logs/ml.googleapis.com%2Ftest_job_GcloudColab_3
projects/nifty-episode-231612/logs/ml.googleapis.com%2Ftest_job_GcloudColab_firse
projects/nifty-episode-231612/logs/ml.googleapis.com%2Ftest_job_GcloudColab_kuchbhi
projects/nifty-episode-231612/logs/ml.googleapis.com%2Ftest_job_GoogleColab
projects/nifty-episode-231612/logs/ml.googleapis.com%2Ftest_job_gcloudColab
projects/nifty-episode-231612/logs/ml.googleapis.com%2Ftest_job_googlecolab
projects/nifty-episode-2316

## Version resource is Created successfully!

**Now to Serve prediction:**
* Encode an image into base64 and add it to a JSON string
* Pass that JSON string to predictor.MyPredictor.predict as in folllowing cell

**Recall predict method of class MyPredictor in predictor module**,
The image instance passed to it as function argument will be decoded in following manner:

* inputs= base64.b64decode(instances['image_bytes']['b64'])

* inputs= scipy.misc.imread(io.BytesIO(inputs))

**Therefore taking cognizance of the decoding at the predict method end, test image for prediction needs to be encoded in accordance as follows.**


In [0]:
import base64
with open('test_img.jpg','rb') as image:
  img_str= base64.b64encode(image.read())
  instances= {'image_bytes': {'b64': base64.b64encode(img_str).decode()}}

* Define Project ID, Model name & version of Serving model.

In [46]:
!gcloud config list

[component_manager]
disable_update_check = True
[core]
account = quantumbisht@gmail.com
project = nifty-episode-231612

Your active configuration is: [default]


In [45]:
import googleapiclient.discovery
service = googleapiclient.discovery.build('ml', 'v1')
PROJECT_ID='nifty-episode-231612'
MODEL_NAME='FoodPredictor_app'
VERSION_NAME='v1_a'

name = 'projects/{}/models/{}/versions/{}'.format(PROJECT_ID, MODEL_NAME, VERSION_NAME)

response = service.projects().predict(name=name,body={'instances': instances}).execute()


W0730 10:16:53.395616 140175376680832 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import Lock

projects/nifty-episode-231612/models/FoodPredictor_app/versions/v1_a
